In [70]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [62]:
!pip install selenium

In [72]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [74]:
URL= "http://isb.nu.edu.pk/Faculty/allfaculty/"

In [172]:
facultyWebdriver = webdriver.Chrome()
facultyWebdriver.get(URL)

In [176]:
faculty = WebDriverWait(facultyWebdriver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "our-team")))
print(len(faculty))

267


In [178]:
ID = []                    # Integer
Name = []                  # String
Designation = []           # String
HEC_Approved_PHD_Supervisor = []  # Bool
Highest_Education = []     # String
Email = []                 # EMAIL
Department = []            # String
Extension = []             # Integer
ImageURL = []              # URL

In [180]:
for mem in faculty:
    name = mem.find_element(By.CLASS_NAME, "title")
    Name.append(name.text)

    designation = mem.find_element(By.CLASS_NAME, "post")
    Designation.append(designation.text)
    
    email = mem.find_element(By.TAG_NAME, "p")
    Email.append(email.text)
    
    image = mem.find_element(By.TAG_NAME, "img")
    ImageURL.append(image.get_attribute("src"))

    id = mem.find_element(By.TAG_NAME, "a")
    ID.append(int(id.get_attribute("href").split("/")[-1]))

In [122]:
print(ID,ImageURL,Name,Email,Designation)

[4553, 4551, 4400, 5099, 6478, 5200, 5002, 3629, 5689, 5724, 5783, 5823, 6259, 9214, 6139, 5830, 5690, 5586, 5643, 4708, 5196, 5249, 6405, 6683, 6867, 7023, 6297, 6735, 6737, 6776, 6975, 6990, 6868, 6804, 6606, 6609, 6579, 6134, 5250, 3949, 4129, 4381, 4401, 4549, 5201, 5425, 5470, 5691, 6032, 5815, 5829, 6079, 6283, 9387, 7013, 7022, 7091, 6866, 9489, 9242, 6296, 6247, 6248, 6252, 6260, 6261, 6263, 6109, 6110, 6111, 6143, 6229, 6243, 6244, 5825, 5793, 6067, 5970, 5976, 5981, 5876, 5880, 5883, 5692, 5693, 5694, 5695, 5704, 5723, 5537, 5465, 5687, 6725, 6696, 6697, 6698, 6701, 6706, 6707, 6610, 6613, 6688, 6438, 6451, 6387, 6508, 6521, 6528, 6529, 6534, 6823, 6842, 6869, 7029, 7030, 6801, 6802, 7006, 7009, 7010, 7075, 7080, 7081, 7083, 6993, 7004, 6795, 6797, 6912, 6913, 6916, 6970, 6972, 6974, 6846, 6848, 6849, 6850, 6851, 6853, 6855, 6856, 6862, 6388, 6384, 6385, 6692, 6693, 6694, 6695, 6651, 6654, 6708, 6245, 5677, 5912, 5922, 5774, 5582, 4154, 5393, 6582, 6527, 6540, 6350, 6351, 484

In [182]:
faculty_driver = webdriver.Chrome()
try:
    for mem in faculty:
        faculty_links = mem.find_elements(By.TAG_NAME, "a")

        for link in faculty_links:
            memberLink = link.get_attribute('href')
            faculty_driver.get(memberLink)

            member = WebDriverWait(faculty_driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "profile-info")))
            spanElements = member.find_elements(By.TAG_NAME, "span")
            memberLabels = member.find_elements(By.TAG_NAME, "label")

            try:
                education = WebDriverWait(faculty_driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='DegreeName']//label")))
                Highest_Education.append(education.text if education.text else "N/A")
            except:
                Highest_Education.append("N/A")

            HEC_Approved_PHD_Supervisor.append(len(memberLabels) > 1)

            try:
                Extension.append(int(spanElements[1].text[-3:]) if len(spanElements) > 1 else -1)
            except (ValueError, IndexError):
                Extension.append(-1)
finally:
    faculty_driver.quit()

In [184]:
departmentDriver = webdriver.Chrome()
departmentDriver.get(URL)
department_mapping = {
    'facultyCS': 'Computing',
    'facultyEE': 'Electrical Engineering',
    'facultyMS': 'Management Sciences',
    'facultySH': 'Sciences and Humanities'
}

In [186]:
for dept_id, name in department_mapping.items():
    depSection = WebDriverWait(departmentDriver, 10).until(EC.presence_of_element_located((By.ID, dept_id)))
    team = WebDriverWait(depSection, 10).until(EC.presence_of_all_elements_located((By.XPATH, ".//*[contains(@class, 'our-team')]")))
    Department.extend([name] * len(team))

In [188]:
print(len(Department))
print(len(Extension))
print(len(HEC_Approved_PHD_Supervisor))
print(len(Highest_Education))

267
267
267
267


In [190]:
faculty_dfISL = pd.DataFrame({'ID': ID , 'Name': Name , 'Designation': Designation , 'HEC_Approved_PHD_Supervisor': HEC_Approved_PHD_Supervisor ,'Highest_Education': Highest_Education , 'Email' : Email , 'Department': Department , 'Extension': Extension,'ImageURL': ImageURL })

In [192]:
faculty_dfISL.to_csv('isl.csv')